In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import os
import glob
import os.path

Using TensorFlow backend.


In [2]:
train_dir = os.path.join('train')
test_dir = os.path.join('test')

imsidir = os.path.join('train')
folder_list = os.listdir(imsidir)
categories = folder_list
folder_list

['IRENE', 'IU', 'JENNY', 'LISA', 'NAYEON', 'ROSE', 'TAEYEON', 'YEJI', 'YOONA']

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(64,64),
                                                   batch_size=720,
                                                   class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                 target_size=(64,64),
                                                 batch_size=1,
                                                 class_mode='categorical')

for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    break

Found 2700 images belonging to 9 classes.
Found 720 images belonging to 9 classes.
배치 데이터 크기: (720, 64, 64, 3)
배치 레이블 크기: (720, 9)


In [13]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(3,3), activation='relu', input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(9, activation='softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 62, 62, 16)        448       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 31, 31, 16)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 29, 29, 64)        9280      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 12, 12, 64)        36928     
_________________________________________________________________
flatten_4 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)              

In [11]:
from keras import backend as K

def precision(y_target, y_pred):
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))
    y_target_yn = K.round(K.clip(y_target, 0, 1)) 

    count_true_positive = K.sum(y_target_yn * y_pred_yn) 
    count_true_positive_false_positive = K.sum(y_pred_yn)

    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    return precision


def recall(y_target, y_pred):
    y_target_yn = K.round(K.clip(y_target, 0, 1))
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) 

    count_true_positive = K.sum(y_target_yn * y_pred_yn) 
    count_true_positive_false_negative = K.sum(y_target_yn)

    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    return recall


def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    return _f1score

In [14]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

history = model.fit_generator(train_generator,
                              steps_per_epoch=10,
                              epochs=25,
                              validation_data=test_generator,
                              validation_steps=100)

Epoch 1/25
10/10 [==============================] - 10s 1s/step - loss: 2.2006 - acc: 0.1288 - val_loss: 2.1708 - val_acc: 0.2400
Epoch 2/25
10/10 [==============================] - 10s 972ms/step - loss: 2.1325 - acc: 0.2236 - val_loss: 1.1680 - val_acc: 0.2300
Epoch 3/25
10/10 [==============================] - 10s 1000ms/step - loss: 1.9282 - acc: 0.2861 - val_loss: 0.3721 - val_acc: 0.3600
Epoch 4/25
10/10 [==============================] - 10s 975ms/step - loss: 1.7911 - acc: 0.3461 - val_loss: 1.2648 - val_acc: 0.4600
Epoch 5/25
10/10 [==============================] - 10s 1s/step - loss: 1.6674 - acc: 0.4104 - val_loss: 0.5249 - val_acc: 0.4100
Epoch 6/25
10/10 [==============================] - 10s 978ms/step - loss: 1.5281 - acc: 0.4704 - val_loss: 0.7296 - val_acc: 0.4800
Epoch 7/25
10/10 [==============================] - 10s 1000ms/step - loss: 1.4342 - acc: 0.5041 - val_loss: 1.4919 - val_acc: 0.5500
Epoch 8/25
10/10 [==============================] - 10s 964ms/step - loss

In [15]:
_loss, _acc = model.evaluate_generator(test_generator, steps=800)
print('loss: {:.3f}, accuracy: {:.3f}'.format(_loss, _acc))

loss: 0.012, accuracy: 0.771


In [18]:
class_idx = list(np.arange(9))
class_label = {}

for i in range(9):
    class_label[class_idx[i]] = categories[i]

y_predict = model.predict_generator(test_generator, steps=720)
predicted_class = np.argmax(y_predict, axis=1)
print(predicted_class)

[1 5 7 1 3 4 8 6 5 5 2 5 6 1 4 8 5 5 3 4 4 3 6 1 1 4 3 8 6 5 4 6 3 8 8 3 2
 1 1 0 2 1 2 0 4 3 5 4 4 0 2 7 3 4 2 1 7 1 1 6 1 5 6 1 3 8 4 8 3 1 0 5 1 7
 2 0 5 1 5 1 3 8 8 8 4 3 4 1 0 6 3 5 5 8 5 2 0 6 0 3 3 0 1 6 4 8 3 4 1 3 3
 7 4 6 6 5 7 2 5 7 8 8 1 1 1 5 3 3 2 1 5 4 2 1 8 4 8 8 1 5 2 5 1 3 4 6 7 2
 4 1 6 8 2 7 3 7 4 7 6 8 6 2 5 1 0 8 4 7 8 7 8 2 6 4 5 8 6 7 4 4 5 5 1 1 2
 2 6 6 7 1 7 5 7 1 0 4 5 3 0 7 1 5 4 2 4 0 7 2 8 3 8 7 1 6 1 6 3 1 3 4 0 4
 1 6 2 4 7 3 6 0 6 4 0 1 6 0 6 7 5 6 6 8 1 7 8 7 3 8 1 8 6 6 1 0 0 5 8 3 6
 7 4 8 4 1 5 7 7 0 2 3 5 5 8 5 6 0 6 4 3 7 7 5 5 4 7 2 3 2 4 8 1 6 4 4 5 8
 3 3 3 7 7 3 0 4 5 1 3 6 1 7 1 3 6 6 5 7 7 4 1 4 6 1 4 0 7 5 7 3 8 1 6 6 7
 0 4 3 3 0 2 1 6 1 7 6 4 4 5 8 0 1 4 2 0 5 0 6 6 4 4 6 7 1 1 2 3 0 5 4 5 4
 3 6 7 0 6 1 7 5 7 1 1 7 1 8 0 5 1 6 1 4 6 0 4 1 3 5 1 6 5 8 1 1 1 4 4 5 5
 4 4 0 5 3 8 2 6 8 8 0 4 8 3 0 7 1 3 3 1 4 3 8 8 3 7 4 7 6 5 1 4 5 4 6 4 3
 5 4 1 8 8 2 6 3 4 2 2 0 0 8 0 3 4 1 5 3 6 1 6 3 6 6 7 0 4 5 7 8 5 3 5 8 0
 6 1 7 7 0 6 5 1 3 5 0 1 